In [1]:
#-----------------------------------------------------------------------------------------------------
# BERT와 sLLM 모델을 이용한 질의응답 서비스 구축 예
# - sLLM_test_embed.ipynb에서 임베딩된 문서들을 이용하여 프롬프트 생성 및 입력하는 예시임.
#
# 질의 응답 시스템 과정
# 문서들 전처리 : 
#    단락별루 분할(\n\n) - 불용어 제거 -문장별루 분할.
# 임베딩 : 
#    kpf-sbert-v1.1로  문장 평균 임베딩벡터 구함 - es에 문장별루 단락text와 평균벡터 저장.
# 프롬프트생성 및 입력 : 
#   검색어 입력(회사:과장일때 휴가 일수는 얼마?)-bert로 임베딩 검색(*스코어가 0.6이상인 경우 체택)-sLLM에 검색된 단락 text를 문맥으로 해서 prompt 구성
#   sLLM에 prompt 입력-응답 결과 출력
#-----------------------------------------------------------------------------------------------------
import os
import random
import numpy as np
import pandas as pd
import time
import random

import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

LOGGER = mlogging(loggername="sllm-test", logfilename='../log/sll-test.txt') # 로그

# param-----------------------------------
SEED = 111
seed_everything(SEED)
DEVICE = GPU_info() # GPU 혹은 CPU
OUT_DIMENSION = 0   # 출력 dimension 128 혹은 0(768)
EMBEDDING_METHOD=1 # 1=평균
NUM_CLUSTERS=10
NUM_CLUSTERS_VARIABLE=False
   
BATCH_SIZE=20
FLOAT_TYPE = 'float16'

# ES 접속
ES_URL = 'http://10.10.4.10:9200/'             # elasticsearch 접속 url
ES_INDEX_NAME = 'mpower_doc_768d_f16'            # elasticsearch 인덱스명
ES_INDEX_FILE = './data/mpower10u_768d_1.json' # 인덱스 생성 파일
SEARCH_SIZE = 5                                # 검색 계수
MIN_SCORE = 1.4                                # 검색 1.4 스코어 이하면 제거

# 임베딩 모델 param
MODEL_PATH = '../data11/model/kpf-sbert-v1.1' #'../data11/model/kpf-sbert-128d-v1'
POLLING_MODE = 'mean'

# sLLM 모델 param
LLM_MODEL_TYPE = 0      # 0=아래 지정한 sLLM 모델, 1=GPT 모델, 2=BARD 모델.
lora_weights:str = '../data11/model/LLM/beomi/KoAlpaca-Polyglot-5.8B-LoRA-qa-moco-1/'      # lora weight 경로
llm_model_path:str ='../data11/model/LLM/beomi/KoAlpaca-Polyglot-5.8B/'     # llm 모델경로(KoAlpaca-Polyglot-5.8B 모델이 가장 속도가 빠르고, 잘 응답하는것 같음.)
uselora_weight = True # Lora 사용하는 경우 True
load_8bit = True       # 8bit 로딩 

# prompt_template(모델에 따라 변경)
PROMPT_DICT = {
    #"prompt_context":("아래 내용을 가지고 질문에 대해 간략히 답변해 주세요\n\n### 내용: {context}\n\n### 질문: {query}\n\n### 답변:"),
    #"prompt_no_context":("질문에 대해 간략히 답변해 주세요\n\n### 질문: {query}\n\n### 답변:")
     
    #"prompt_context":("### 질문: {query}\n\n아래 내용을 가지고 질문에 대해 간략히 답변해 주세요\n\n### 내용: {context}\n\n### 답변:"),
    #"prompt_no_context":("### 질문: {query}\n\n질문에 대해 간략히 답변해 주세요\n\n### 답변:")
    
    #"prompt_context":("### 질문: {context}\n\n위 내용을 간략히 요약해 주세요\n\n### 답변:"),
    #"prompt_no_context":("### 질문: {context}\n\n위 내용을 간략히 요약해 주세요\n\n### 답변:")
    
    #"prompt_context":("{context}\n위 내용을 간략히 요약해 주세요\n\n"),
    #"prompt_no_context":("{context}\n위 내용을 간략히 요약해 주세요\n\n"),
    
    #"prompt_context":("다음 내용을 파악하여 질문에 대해 간략한 답변을 작성하세요.\n### 내용: {context}\n### 질문: {query}\n### 답변:"),
    #"prompt_no_context":("질문에 대해 간략한 답변을 작성하세요.\n### 질문: {query} ### 답변:")
    
    #"prompt_context":("### 지시: [질의]에 가장 적합한 [응답]을, 제시된 [문단]에서 찾아서 간단한 문장으로 답변해주세요.\n\n### [질의]: {query}\n\n### [문단]: {context}\n\n### [응답]:"),
    #"prompt_no_context":("### 지시: 질의에 대해, 간단한 문장으로 응답해 주세요 ### [질의]: {query}\n\n### [응답]:")
    
    #"prompt_context":("### 지시: [질의]에 가장 적합한 [응답]을, 제시된 [문단]을 바탕으로 문장으로 작성해 주세요.\n\n### [질의]: {query}\n\n### [문단]: {context}\n\n### [응답]:"),
    #"prompt_no_context":("### 지시: 질의에 대해, 자세하게 문장으로 작성해 주세요\n\n### [질의]: {query}\n\n### [응답]:")
    
    "prompt_context":("###지시: 문단에서 질의에 대해 가장 적합한 내용을 찾아, 응답 문장을 만들어 주세요.\n\n###[문단]: {context}\n\n###[질의]: {query}\n\n###[응답]:"),
    "prompt_no_context":("###지시: 질의에 대해, 자세하게 응답 문장을 만들어 주세요.\n\n###[질의]: {query}\n\n###[응답]:")
}
#-------------------------------------------

# prompt 테스트 
print(f'\n\nprompt 테스트----------------------\n')
query='제주도의 크기는 얼마?'
#context = ''
context = '제주도는 최남단에 있는 섬으로, 길이는 40km가 되는 대한민국에서 가장큰 섬이다.'
if context:
    prompt = PROMPT_DICT['prompt_context'].format(query=query, context=context)
else:
    prompt = PROMPT_DICT['prompt_no_context'].format(query=query)
    
print(prompt)

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


logfilepath:../log/sll-test.txt_2023-06-29.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


prompt 테스트----------------------

###지시: 문단에서 질의에 대해 가장 적합한 내용을 찾아, 응답 문장을 만들어 주세요.

###[문단]: 제주도는 최남단에 있는 섬으로, 길이는 40km가 되는 대한민국에서 가장큰 섬이다.

###[질의]: 제주도의 크기는 얼마?

###[응답]:


In [ ]:
'''
import re

response = """
### 지시: 답변\n\n### [문단]: [출장 여비 규정]\n4.1. 일수 계산 시, 기산은 오전 0시부터로 하며 1일 미만의 단수는 1일로 계산한다.\n\n### [질의]: 1일 출장시 숙박비는 얼마?\n\n### [응답]: 1일 출장시 숙박비는 1일당 실비로 지원합니다.
"""

context:str = ""
if response:
    answers = response.split("###[응답]:")
        
    # [문단]부터 [질의]까지 추출
    pattern = r"\[문단\]: (.+?)(?=\n\n### \[질의\])"
    context = re.search(pattern, response, re.DOTALL).group(1)

print(context)
'''

In [2]:
#----------------------------------------------------------------------
# 임베딩 모델 로딩
#----------------------------------------------------------------------
from tqdm.notebook import tqdm
from myutils import embed_text, bi_encoder, mpower_index_batch

TOKENIZER1, BI_ENCODER1 = bi_encoder(model_path=MODEL_PATH, max_seq_len=512, do_lower_case=True, pooling_mode=POLLING_MODE, out_dimension=OUT_DIMENSION, device=DEVICE)

print(BI_ENCODER1)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [3]:
#----------------------------------------------------------------------
# sLLM 모델 로딩
#----------------------------------------------------------------------
import torch
import transformers
from peft import PeftModel
import time

if LLM_MODEL_TYPE == 0:     # 0=아래 지정한 sLLM 모델, 1=GPT 모델, 2=BARD 모델. == True:
    start_time = time.time()

    # tokenizer 로딩
    tokenizer = transformers.AutoTokenizer.from_pretrained(llm_model_path)

    # 원본 모델 로딩
    model = transformers.AutoModelForCausalLM.from_pretrained(llm_model_path, load_in_8bit=load_8bit, torch_dtype=torch.float16, device_map="auto")

    if uselora_weight:
        model = PeftModel.from_pretrained(model, lora_weights, torch_dtype=torch.float16) # loRA 모델 로딩

    if not load_8bit:
        model.half()

    model.eval()

    end_time = time.time() - start_time
    print("time: {:.2f} ms\n".format(end_time * 1000)) 
    print(model)
    


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /MOCOMSYS/anaconda3/envs/bong/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/MOCOMSYS/anaconda3/envs/bong/lib/libcudart.so'), PosixPath('/MOCOMSYS/anaconda3/envs/bong/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
[Kss]: The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

time: 39933.52 ms

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 4096)
        (layers): ModuleList(
          (0-27): 28 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear8bitLt(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=12288, bias=Fa

In [4]:
# ES 관련
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk

#--------------------------------------------
# 조건에 맞게 임베딩 처리하는 함수 
#--------------------------------------------
def embedding(paragraphs:list)->list:
    # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
    embeddings = embed_text(model=BI_ENCODER1, paragraphs=paragraphs, return_tensor=False).astype(FLOAT_TYPE)    
    return embeddings

#------------------------------------------------------------------------
# ES임베딩 쿼리 벡터
# -쿼리 임베딩 벡터를 구하고, es에 접속해서 rfile_text 뽑아냄
#------------------------------------------------------------------------
def es_embed_query(query:str):
    assert query, f'query is empty'
    
    # 1.elasticsearch 접속
    es = Elasticsearch(ES_URL)  

    # 임베딩 구함.
    start_time = time.time()
    embed_query = embedding([query])[0]
    
    #print(len(embed_query))
    
    # 쿼리 구성
    script_query = {
        "script_score":{
            "query":{
                "match_all": {}},
            "script":{
                "source": "cosineSimilarity(params.query_vector, doc['vector1']) + 1.0",  # 뒤에 1.0 은 코사인유사도 측정된 값 + 1.0을 더해준 출력이 나옴
                "params": {"query_vector": embed_query}
            }
        }
    }
    
    #print(script_query)

    # 실제 ES로 검색 쿼리 날림
    start_search_time = time.time()
    response = es.search(
        index=ES_INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source":{"includes": ["rfile_name", "rfile_text"]}
        }
    )
    end_time = time.time() - start_time
    print("*ES 검색시간: {:.2f} ms".format(end_time * 1000)) 

    count = 0
    docs = []
    for hit in response["hits"]["hits"]: 
        doc = {}  #dict 선언
        doc['rfile_name'] = hit["_source"]["rfile_name"]      # contextid 담음
        doc['rfile_text'] = hit["_source"]["rfile_text"]      # text 담음.
        doc['score'] = hit["_score"]
        docs.append(doc) 
            
    return docs
#------------------------------------------------------------------------


In [5]:
#--------------------------------------------
# prompt 생성
#--------------------------------------------
def make_prompt(docs, query)->str:
     # prompt 구성
    context:str = ''

    for doc in docs:
        score = doc['score']
        if score > MIN_SCORE:
            rfile_text = doc['rfile_text']
            if rfile_text:
                context += rfile_text + '\n\n'
                
    if context:
        prompt = PROMPT_DICT['prompt_context'].format(query=query, context=context)
    else:
        prompt = PROMPT_DICT['prompt_no_context'].format(query=query)
                
    # KoAlpaca 프롬프트
    #prompt = f"### 질문: {query}\n질문에 대해 아래 내용을 바탕으로 간략히 답변해 주세요\n\n### 문맥: {context}\n\n### 답변:" if context else f"### 질문: {query}\n질문에 대해 간략히 답변해 주세요\n\n### 답변:"
    
    # llama 프롬프트
    #prompt = f"아래는 작업을 설명하는 명령어입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### Instruction: {context}\n{query} ### Response:"

    #print(prompt)
    #print()
    return prompt
    


In [6]:
from transformers import GenerationConfig
#-----------------------------------------
# sLLM 이용한 text 생성
#-----------------------------------------
def generate_text_sLLM(prompt):
    
    max_new_tokens = 256
    eos_str = tokenizer.decode(tokenizer.eos_token_id)
    start_time = time.time()
    #print(f'eos_str:{eos_str}')
    
    #prompt = query
    #prompt = f"### 질문: {input_text}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {input_text}\n\n### 답변:"
    #prompt = f"### 질문 : 간략히 답변해줘.{query}\r\n###답변:"
    #prompt = f"### 질문: {query}\n\n### 답변:"
    #print(prompt)

    # config 설정
    generation_config = GenerationConfig(
        temperature=0.5,
        top_p=0.75,
        top_k=40,
        num_beams=1,
        bos_token_id=tokenizer.bos_token_id,  # 시작토큰 
        eos_token_id=tokenizer.eos_token_id,  # end 토큰
        pad_token_id=tokenizer.pad_token_id   # padding 토큰
    )

    # 프롬프트 tokenizer 
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    input_ids = inputs["input_ids"]
    #print(input_ids)
       
    # Without streaming
    # generate 처리
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=False,
            max_new_tokens=max_new_tokens,
        )

    # 출력
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)

    end_time = time.time() - start_time
    print("*Text생성시간: {:.2f} ms\n".format(end_time * 1000)) 
    #print(output.replace(eos_str, ''))
    #print()
    return output.replace(eos_str, '')


In [7]:
#-----------------------------------------
# GPT를 이용한 text 생성
#-----------------------------------------
import openai

# key 지정
openai.api_key = "sk-xxxx"

# 모델 - GPT 3.5 Turbo 지정
# => 모델 목록은 : https://platform.openai.com/docs/models/gpt-4 참조
gpt_model = "gpt-3.5-turbo"#"gpt-4"#"gpt-3.5-turbo" #gpt-4-0314
'''
ROLE_PROMPT = "아래 내용을 요약해서 질문에 알맞게 간략히 답변해주세요"

# 메시지 설정
MESSAGES = [
            {"role": "system", "content": "You are a helful assistant."},
            #{"role": "system", "content": "질문에 대해 요약해줘"},
            #{"role": "user", "content" : "How are you?"},
            #{"role": "assistant", "content" : "I am doing well"},
            {"role": "user", "content": ROLE_PROMPT}
        ]
'''
MESSAGES:list = []
#-----------------------------------------
# GPT를 이용한 text 생성
#-----------------------------------------
def generate_text_GPT(prompt, messages):
    
    print(f'len(messages):{len(messages)}') 
    print()
    
    #-------------------------------------------------------
    # *** gpt에 메시지는 계속 대화 내용이 유지가 되므로, 비용이 발생함.
    # 따라서 최근 2개 대화만 유지함.
    #if len(messages) >= 2:
    #    messages = messages[len(messages)-2:]  # 최근 2개의 대화만 가져오기
    messages = []  # 무조건 최근대화 초기화
     #-------------------------------------------------------
        
    # 사용자 메시지 추가
    messages.append( {"role": "user", "content": prompt})
    print(messages)

    # ChatGPT-API 호출하기
    response = openai.ChatCompletion.create(
        model=gpt_model,
        messages=messages,
        max_tokens=500, # 토큰 수 
        temperature=1,  # temperature 0~2 범위 : 작을수록 정형화된 답변, 클수록 유연한 답변(2는 엉뚱한 답변을 하므로, 1.5정도가 좋은것 같음=기본값은=1)
        top_p=0.1 # 기본값은 1 (0.1이라고 하면 10% 토큰들에서 출력 토큰들을 선택한다는 의미)
    )

    print(response)
    print()
    answer = response['choices'][0]['message']['content']
    return answer

In [8]:
#------------------------------------------------------------------
# 구글 bard를 이용한 text 생성
#
# 세션키를 이용하여 구글 bard 테스트 예제
# 출처 : https://github.com/dsdanielpark/Bard-API
#
# token 값얻기
# https://bard.google.com/ 방문
# 콘솔용 F12
# 세션: 애플리케이션 → 쿠키 → 쿠키 값을 복사합니다 __Secure-1PSID.
# -> 참고로 반드시 뒤에 .으로 끝나고 .포함해서 길이가 72자임.
#------------------------------------------------------------------

from bardapi import Bard

token = 'XQhPmzE3Wa_GqgDH1Z9YcRwZieE0STZi0ANZ557Zcm9Lio8QeIQtQvdd8evImbUrF-ZapQ.' # bard 토큰 입력

def generate_text_BARD(prompt):
    
    bard = Bard(token=token,timeout=30) # Set timeout in seconds
    answer = bard.get_answer(prompt)['content']
    return answer

In [9]:
# 쿼리 입력후 테스트
def run_query_loop():
    while True:
        try:
            handle_query()
        except KeyboardInterrupt:
            return

def handle_query():
    # 쿼리는 일반 쿼리인 경우 '일반##제주도 면적은 얼마?'
    #query = '일반##제주도 면적은 얼마?' 
    query = input("질문을 입력하세요")
    
    query_split = query.split('##')
    prefix = query_split[0]
    #print(f'prefix: {prefix}')
    
    if prefix == '@':  # 일반쿼리일때는 @## prefix 입력후 질문입력함. 
        query1 = query_split[1]
        prompt = make_prompt(docs='', query=query1)
    else:
        query1 = query
        docs = es_embed_query(query1)
        print(docs)
        print()
        prompt = make_prompt(docs=docs, query=query1)
        
    if LLM_MODEL_TYPE == 0:     # 0=아래 지정한 sLLM 모델, 1=GPT 모델, 2=BARD 모델
        print(generate_text_sLLM(prompt))
    elif LLM_MODEL_TYPE == 1:     # 0=아래 지정한 sLLM 모델, 1=GPT 모델, 2=BARD 모델:
        print(generate_text_GPT(prompt=prompt, messages=MESSAGES))
    else:
        print(generate_text_BARD(prompt))
        
    print()
              

In [ ]:
'''
PROMPT_DICT = {   
    #"prompt_context":("###지시: 문단에서 질의에 대해 가장 적합한 내용들을 찾고, 찾은 내용을 질의에 적합하게 요약해서 답변해 주세요\n\n###[문단]: {context}\n\n###[질의]: {query}\n\n###[응답]:"),
    #"prompt_no_context":("###지시: 질의에 대해, 자세하게 문장으로 작성해 주세요\n\n### [질의]: {query}\n\n###[응답]:")
    
    "prompt_context":("###지시: 문단에서 질의에 대해 가장 적합한 내용을 찾아, 응답 문장을 만들어 주세요.\n\n###[문단]: {context}\n\n###[질의]: {query}\n\n###[응답]:"),
    "prompt_no_context":("###지시: 질의에 대해, 자세하게 응답 문장을 만들어 주세요.\n\n###[질의]: {query}\n\n###[응답]:")
}
'''
run_query_loop()

질문을 입력하세요 출장시 숙박비는 얼마?


/tmp/ipykernel_218012/813833492.py:45: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es.search(
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(

*ES 검색시간: 1477.94 ms
[{'rfile_name': 19, 'rfile_text': '출장 여비 규정\n13. 출장중 상병\n임직원이 출장중 부상 또는 질병으로 가족의 간호가 필요하다고 인정될 경우 가족 1인에 대하여 왕복여비로 본인과 동일등급의 여비상당액을 지급하며, 의사의 진단서 등 이를 증명할 수 있는 증빙서류에 의하여 연장체제중의 여비를 지급할 수 있다.', 'score': 1.5547521}, {'rfile_name': 19, 'rfile_text': '출장 여비 규정\n10. 여비지급 및 정산\n10.1 여비는 출장전 여비 지급기준에 의거 지급함을 원칙으로 한다.\n10.2 출장지의 교통 편의상 택시를 타야 할 경우, 출장여비신청서 기안시에 명기하여 사전 승인을 듣하여야 하며, 법인카드를 사용하거나 추후 정산 받을 수 있다.\n10.3 용무를 마치고 귀임한 때에는 지체없이 출장결과를 서면으로 보고하여야 한다.\n단, 국내출장의 경우 경미한 사항이나 단순한 업무 또는 기밀에 속하는 사항에 대하여는 구두로 보고할 수 있다.\n10.4 출장시 사전에 승인을 득하지 않은 경우, 일비, 식비의 명목으로 법인카드 사용 불가한다. (접대 등 필요시 출장신청서에 표기)', 'score': 1.5502534}, {'rfile_name': 19, 'rfile_text': '출장 여비 규정\n12. 휴직자, 해직자에 대한 여비\n사무인계 또는 잔무정리등을 위하여 휴직 또는 해직직원 출장시 본직 또는 전직에 상당한 여비를 지급한다.', 'score': 1.5428421}, {'rfile_name': 19, 'rfile_text': '출장 여비 규정\n3. 여비계산\n3.1. 여비는 순로에 의하여 계산하며, 업무 형편상 또는 부득이한 사유로 순로를 변경하였을 경우 제 경유한 노정에 의해 계산한다.\n3.2. 근무지 또는 출장지 외의 곳에 거주하거나 체재하는 자가 그 거주지 도는 체재지로부터 목적지까지 직접 여행하는 경우에는 그 곳에서 목적지에 이르는 여비를 

질문을 입력하세요 1일 출장시 숙박비는 얼마인가요?


*ES 검색시간: 66.32 ms
[{'rfile_name': 19, 'rfile_text': '출장 여비 규정\n국내출장여비 1\n부장 이하: 철도 운임은 실비(KTX일반), 선박운임은 실비(2등급), 항공운임은 실비(일반실), 자동차 운임은 실비로 지원함.\n부장 이하: 1일비는 20,000원 지원함.\n부장 이하: 1일 숙박비는 1박당 실비로 지원함. 단 서울특별시는 사항액이 70,000원, 광역시는 60,000원, 그 밖의 지역은 50,000원 임\n부장 이하: 1일 식비는 20,000원 지원함.\n임원 이상: 철도 운임은 실비(KTX특실), 선박운임은 실비(1등급), 항공운임은 실비(일반실), 자동차 운임은 실비로 지원함.\n임원 이상: 1일비는 20,000원 지원함.\n임원 이상: 1일 숙박비는 1박당 실비로 지원함.\n임원 이상: 1일 식비는 25,000원 지원함.', 'score': 1.5027854}, {'rfile_name': 19, 'rfile_text': '출장 여비 규정\n4. 지급기준\n4.1. 일수 계산 시, 기산은 오전 0시부터로 하며 1일 미만의 단수는 1일로 계산한다.\n숙박비는 숙박일수에 따라 계산하며, 기산은 오전 0시를 경과함으로써 1박으로 하며, 숙박이 다른 출장목적지로의 이동 중 교통편 내에서 이루어졌을 경우 여행 목적지의 해당 숙박비만을 지급한다.', 'score': 1.4711107}, {'rfile_name': 19, 'rfile_text': '출장 여비 규정\n6. 일비, 숙박비 및 식비\n6.1. 일비, 숙박비, 식비의 지급\n6.1.1. 국내 여행자의 일비, 숙박비 및 식비는 별표 1에 따라 지급하고, 국외 여행자의 경우는 별표2에 따라 지급한다. \n다만, 부득이한 사유로 숙박비의 상한을 초과하는 여비 지출이 필요한 경우, 국내 여행의 경우에는 숙박비 상한액의 10분의 3을, 국외 여행의 경우 2분의 1을 넘지 아니하는 범위에서 여비를 추가로 지급할 수 있다. \n이 경우, 국내 여행의 경우, 본부장의 사전

...

*ES 검색시간: 56.45 ms
[{'rfile_name': 19, 'rfile_text': '출장 여비 규정\n국내출장여비 1\n부장 이하: 철도 운임은 실비(KTX일반), 선박운임은 실비(2등급), 항공운임은 실비(일반실), 자동차 운임은 실비로 지원함.\n부장 이하: 1일비는 20,000원 지원함.\n부장 이하: 1일 숙박비는 1박당 실비로 지원함. 단 서울특별시는 사항액이 70,000원, 광역시는 60,000원, 그 밖의 지역은 50,000원 임\n부장 이하: 1일 식비는 20,000원 지원함.\n임원 이상: 철도 운임은 실비(KTX특실), 선박운임은 실비(1등급), 항공운임은 실비(일반실), 자동차 운임은 실비로 지원함.\n임원 이상: 1일비는 20,000원 지원함.\n임원 이상: 1일 숙박비는 1박당 실비로 지원함.\n임원 이상: 1일 식비는 25,000원 지원함.', 'score': 1.4908354}, {'rfile_name': 19, 'rfile_text': '출장 여비 규정\n4. 지급기준\n4.1. 일수 계산 시, 기산은 오전 0시부터로 하며 1일 미만의 단수는 1일로 계산한다.\n숙박비는 숙박일수에 따라 계산하며, 기산은 오전 0시를 경과함으로써 1박으로 하며, 숙박이 다른 출장목적지로의 이동 중 교통편 내에서 이루어졌을 경우 여행 목적지의 해당 숙박비만을 지급한다.', 'score': 1.4844524}, {'rfile_name': 19, 'rfile_text': '출장 여비 규정\n6. 일비, 숙박비 및 식비\n6.1. 일비, 숙박비, 식비의 지급\n6.1.1. 국내 여행자의 일비, 숙박비 및 식비는 별표 1에 따라 지급하고, 국외 여행자의 경우는 별표2에 따라 지급한다. \n다만, 부득이한 사유로 숙박비의 상한을 초과하는 여비 지출이 필요한 경우, 국내 여행의 경우에는 숙박비 상한액의 10분의 3을, 국외 여행의 경우 2분의 1을 넘지 아니하는 범위에서 여비를 추가로 지급할 수 있다. \n이 경우, 국내 여행의 경우, 본부장의 사전

질문을 입력하세요 1일 출장시 식비는 얼마인가요?


*ES 검색시간: 60.57 ms
[{'rfile_name': 19, 'rfile_text': '출장 여비 규정\n6. 일비, 숙박비 및 식비\n6.1. 일비, 숙박비, 식비의 지급\n6.1.1. 국내 여행자의 일비, 숙박비 및 식비는 별표 1에 따라 지급하고, 국외 여행자의 경우는 별표2에 따라 지급한다. \n다만, 부득이한 사유로 숙박비의 상한을 초과하는 여비 지출이 필요한 경우, 국내 여행의 경우에는 숙박비 상한액의 10분의 3을, 국외 여행의 경우 2분의 1을 넘지 아니하는 범위에서 여비를 추가로 지급할 수 있다. \n이 경우, 국내 여행의 경우, 본부장의 사전 승인을 득해야 하고, 국외 여행의 경우, 대표이사의 사전 승인을 득해야 한다.\n6.1.2. 일비는 여행일수에 따라 지급하되, 법인차량을 이용하는 경우에는 일비의 2분의 1을 지급한다.\n6.1.3. 식비는 여행일수에 따라 지급한다. 다만, 수로여행과 항공여행에 는 따로 식비가 필요한 경우에만 식비를 지급한다.', 'score': 1.5438462}, {'rfile_name': 19, 'rfile_text': '출장 여비 규정\n국내출장여비 1\n부장 이하: 철도 운임은 실비(KTX일반), 선박운임은 실비(2등급), 항공운임은 실비(일반실), 자동차 운임은 실비로 지원함.\n부장 이하: 1일비는 20,000원 지원함.\n부장 이하: 1일 숙박비는 1박당 실비로 지원함. 단 서울특별시는 사항액이 70,000원, 광역시는 60,000원, 그 밖의 지역은 50,000원 임\n부장 이하: 1일 식비는 20,000원 지원함.\n임원 이상: 철도 운임은 실비(KTX특실), 선박운임은 실비(1등급), 항공운임은 실비(일반실), 자동차 운임은 실비로 지원함.\n임원 이상: 1일비는 20,000원 지원함.\n임원 이상: 1일 숙박비는 1박당 실비로 지원함.\n임원 이상: 1일 식비는 25,000원 지원함.', 'score': 1.4881005}, {'rfile_name': 19, 'rfile_text': '출장

질문을 입력하세요 제주에서 서울로 1박2일 출장시 숙박비와 실비는 얼마인가요?


*ES 검색시간: 66.71 ms
[{'rfile_name': 19, 'rfile_text': '출장 여비 규정\n국내출장여비 1\n부장 이하: 철도 운임은 실비(KTX일반), 선박운임은 실비(2등급), 항공운임은 실비(일반실), 자동차 운임은 실비로 지원함.\n부장 이하: 1일비는 20,000원 지원함.\n부장 이하: 1일 숙박비는 1박당 실비로 지원함. 단 서울특별시는 사항액이 70,000원, 광역시는 60,000원, 그 밖의 지역은 50,000원 임\n부장 이하: 1일 식비는 20,000원 지원함.\n임원 이상: 철도 운임은 실비(KTX특실), 선박운임은 실비(1등급), 항공운임은 실비(일반실), 자동차 운임은 실비로 지원함.\n임원 이상: 1일비는 20,000원 지원함.\n임원 이상: 1일 숙박비는 1박당 실비로 지원함.\n임원 이상: 1일 식비는 25,000원 지원함.', 'score': 1.460317}, {'rfile_name': 19, 'rfile_text': '출장 여비 규정\n4. 지급기준\n4.1. 일수 계산 시, 기산은 오전 0시부터로 하며 1일 미만의 단수는 1일로 계산한다.\n숙박비는 숙박일수에 따라 계산하며, 기산은 오전 0시를 경과함으로써 1박으로 하며, 숙박이 다른 출장목적지로의 이동 중 교통편 내에서 이루어졌을 경우 여행 목적지의 해당 숙박비만을 지급한다.', 'score': 1.4505268}, {'rfile_name': 19, 'rfile_text': '출장 여비 규정\n국내출장여비 2\n임원은 일비와 식비를 실비로 정산할 수 있다.\n자가용 승용차를 이용할 경우의 운임은 유류비와 통행료를 제공한다. \n유류비는 이동거리를 산정하여, 1L당 8km, 오피넷 서울지역 평균 휘발류 가격을 기준으로 산정한다. \n산정된 유류비를 법인카드로 결제하여 사용한다. \n통행료는 실비 지원하고, 법인카드를 통해 결제하거나 영수증을 통해 추후 청구할 수 있다. \n차량 이용시 별도의 일비는 지급하지 않는다.\n법인차량을 이용할 경우, 운임과